In [2]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
print(volume.GetVolumeRange())
volume.SetMasterVolumeLevel(-20, None)


(-65.25, 0.0, 0.03125)


0

In [3]:
import cv2
import numpy as np
import time
import math
import screen_brightness_control as sbc

In [4]:
from ipynb.fs.full.HandTrackingModule import HandDetector

In [10]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
detector = HandDetector()
radius1 = 100
radius2 = 100
radius3 = 100
colorLevel = 100
while True:
    success,frame = cap.read()
    if not success:
        break
    frame = cv2.flip(frame,1)
    hands,frame = detector.findHands(frame)
    if hands:
        hand1 = hands[0]
        lmList = hand1["lmList"]
        x1,y1 = lmList[4][0],lmList[4][1]
        x2,y2 = lmList[8][0],lmList[8][1]
        cv2.circle(frame,(x1,y1),10,(200,0,200),-1)
        cv2.circle(frame,(x2,y2),10,(200,0,200),-1)
        cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),2)
        length = math.hypot((x1-x2),(y1-y2))
        #range from 20 to 500
        if length >= 0 and length <= 50:
            color1,color2,color3 = 155,155,155
        elif length >= 50 and length <= 150:
            color1,color2,color3 = 155,155,255
        elif length >= 150 and length <= 350 :
            color1,color2,color3 = 155,255,255
        else:
            color1,color2,color3 = 255,255,255
        volumeLevel = np.interp(length,(20,320),(-65.25,0))
        brightnessLevel = np.interp(length,(0,320),(0,100))
        colorLevel = np.interp(length,(20,320),(0,255))
        volume.SetMasterVolumeLevel(volumeLevel, None)
        sbc.set_brightness(int(brightnessLevel))
        
        draw_ellipse(frame,80, (200,600), (80,80), 175, 120, 250, (color1,color1,color1),4)
        draw_ellipse(frame,50, (170,600), (80,80), 175, 120, 250, (color2,color2,color2),4)
        draw_ellipse(frame,20, (140,600), (80,80), 175, 120, 250, (color3,color3,color3),4)
        
        #cv2.putText(frame,str(int(length)),(50,50),4,2,(0,0,0),2)
        cv2.circle(frame,(200,200),int(100),(0,int(colorLevel),int(colorLevel)),3)
        cv2.circle(frame,(200,200),int(70),(0,int(colorLevel),int(colorLevel)),3)
        cv2.circle(frame,(200,200),int(40),(0,int(colorLevel),int(colorLevel)),3)
    

    #cv2.ellipse(frame, center, axes, angle, startAngle, endAngle, color,2)
    
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [1]:
def draw_ellipse(frame,radius, center, axes, angle, startAngle, endAngle, color,thickness):
    r=radius
    ax = (int(radius),int(radius))
    ang=angle;
    sAng=startAngle;
    eAng=endAngle;
    cent=center
    c=color

    cv2.ellipse(frame, cent, ax, ang, sAng, eAng, c,thickness)
    

In [61]:
sbc.set_brightness(0)